In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from env import get_db_url
import os
from itertools import combinations
# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import wrangle_zillow as wrangle


In [2]:
df = wrangle.get_zillow_data()

In [3]:
df.shape

(52442, 63)

In [4]:
df = wrangle.wrangle_zillow()

In [5]:
df.shape

(30299, 31)

In [ ]:
df = wrangle.handle_missing_values(df, prop_required_column = .9, prop_required_row = .9)

In [ ]:
df.shape

In [ ]:
df = wrangle.handle_outliers2()

In [ ]:
df.logerror.min()

In [ ]:
df.calculatedfinishedsquarefeet.max()

In [ ]:
df.bedroomcnt.max()

In [ ]:
def get_zillow_data():
    filename = 'zillow_data.csv'
    
    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col = 0)
    else:
        df = pd.read_sql(
        '''
        SELECT *
        FROM properties_2017
        JOIN propertylandusetype 
        USING (propertylandusetypeid)
        JOIN predictions_2017
        USING (parcelid)
        WHERE propertylandusedesc = 'Single Family Residential';
        '''
        ,
        get_db_url('zillow')
        )
        
        df.to_csv(filename)
        
        return df

In [ ]:
df = get_zillow_data()
df_original = df.copy()
df_original.shape

In [ ]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    threshold = int(round(prop_required_column * len(df.index), 0))
    df.dropna(axis=1, thresh = threshold, inpolace = True)
    threshold = int(round(prop_requred_row * len(df.columns), 0))
    df.dropna(axis = 0, thresh = threshold, inplace = True)

In [ ]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df


In [ ]:
df = handle_missing_values(df, prop_required_column = .8, prop_required_row = .9)

In [ ]:
52349/52442

In [ ]:
df.isna().sum()

In [ ]:
cols = [
 'bathroomcnt',
 'bedroomcnt',
 'calculatedbathnbr',
 'calculatedfinishedsquarefeet',
 'fips',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidzip',
 'roomcnt',
 'yearbuilt',
 'structuretaxvaluedollarcnt',
 'taxvaluedollarcnt',
 'landtaxvaluedollarcnt',
 'taxamount',
 'id.1',
 'logerror']

In [ ]:
def handle_outliers(df, cols, k):
    # Create placeholder dictionary for each columns bounds
    bounds_dict = {}

    # get a list of all columns that are not object type
    non_object_cols = df.dtypes[df.dtypes != 'object'].index


    for col in non_object_cols:
        # get necessary iqr values
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        upper_bound =  q3 + k * iqr
        lower_bound =  q1 - k * iqr

        #store values in a dictionary referencable by the column name
        #and specific bound
        bounds_dict[col] = {}
        bounds_dict[col]['upper_bound'] = upper_bound
        bounds_dict[col]['lower_bound'] = lower_bound

    for col in non_object_cols:
        #retrieve bounds
        col_upper_bound = bounds_dict[col]['upper_bound']
        col_lower_bound = bounds_dict[col]['lower_bound']

        #remove rows with an outlier in that column
        df = df[(df[col] < col_upper_bound) & (df[col] > col_lower_bound)]
        return df

In [ ]:
handle_outliers(df, cols, 1.5)

In [ ]:
df.shape

In [ ]:
df2=df.copy()

In [ ]:
df2 = handle_outliers(df, cols, 1.15)

In [ ]:
df2.shape

In [ ]:
49328/52349

In [ ]:
df = wrangle.get_zillow_data()

In [ ]:
q1 = df.calculatedfinishedsquarefeet.quantile(0.25)
q3 = df.calculatedfinishedsquarefeet.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 + 1.5 * iqr
lower_bound =  q1 - 1.5 * iqr
print('Q1 = ' , q1 , 'Q3 =' , q3 , 'lower bound = ', lower_bound , 'upper bound = ', upper_bound)
print ('actual min = ',df.calculatedfinishedsquarefeet.min())
print ('actual max =', df.calculatedfinishedsquarefeet.max())

In [ ]:
q1 = df.logerror.quantile(0.25)
q3 = df.logerror.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 + 1.5 * iqr
lower_bound =  q1 - 1.5 * iqr
print('Q1 = ' , q1 , 'Q3 =' , q3 , 'lower bound = ', lower_bound , 'upper bound = ', upper_bound)
print ('actual min = ',df.logerror.min())
print ('actual max =', df.logerror.max())

In [ ]:
q1 = df.bedroomcnt.quantile(0.25)
q3 = df.bedroomcnt.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 + 1.15 * iqr
lower_bound =  q1 - 1.15 * iqr
print('Q1 = ' , q1 , 'Q3 =' , q3 , 'lower bound = ', lower_bound , 'upper bound = ', upper_bound)
print ('actual min = ',df.bedroomcnt.min())
print ('actual max =', df.bedroomcnt.max())

In [ ]:
q1 = df.bathroomcnt.quantile(0.25)
q3 = df.bathroomcnt.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 + 1.15 * iqr
lower_bound =  q1 - 1.15 * iqr
print('Q1 = ' , q1 , 'Q3 =' , q3 , 'lower bound = ', lower_bound , 'upper bound = ', upper_bound)
print ('actual min = ',df.bathroomcnt.min())
print ('actual max =', df.bathroomcnt.max())

In [ ]:
q1 = df.lotsizesquarefeet.quantile(0.25)
q3 = df.lotsizesquarefeet.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 + 1.5 * iqr
lower_bound =  q1 - 1.5 * iqr
print('Q1 = ' , q1 , 'Q3 =' , q3 , 'lower bound = ', lower_bound , 'upper bound = ', upper_bound)
print ('actual min = ',df.lotsizesquarefeet.min())
print ('actual max =', df.lotsizesquarefeet.max())

In [ ]:
q1 = df.taxvaluedollarcnt.quantile(0.25)
q3 = df.taxvaluedollarcnt.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 - 1.15 * iqr
lower_bound =  q1 + 1.15 * iqr
print('Q1 = ' , q1 , 'Q3 =' , q3 , 'lower bound = ', lower_bound , 'upper bound = ', upper_bound)
print ('actual min = ',df.taxvaluedollarcnt.min())
print ('actual max =', df.taxvaluedollarcnt.max())

In [ ]:
df = wrangle.wrangle_zillow()

In [ ]:
df.shape

In [ ]:
q1 = df.logerror.quantile(0.25)
q3 = df.logerror.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 + 1.15 * iqr
lower_bound =  q1 - 1.15 * iqr

In [ ]:
q1 = df.logerror.quantile(0.25)
q3 = df.logerror.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 + 1.15 * iqr
lower_bound =  q1 - 1.15 * iqr
print('Q1 = ' , q1 , 'Q3 =' , q3 , 'lower bound = ', lower_bound , 'upper bound = ', upper_bound, 'iqr = ' , iqr)
print ('actual min = ',df.logerror.min())
print ('actual max =', df.logerror.max())

In [ ]:
q1 = df.bedroomcnt.quantile(0.25)
q3 = df.bedroomcnt.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 + 1.15 * iqr
lower_bound =  q1 - 1.15 * iqr
print('Q1 = ' , q1 , 'Q3 =' , q3 , 'lower bound = ', lower_bound , 'upper bound = ', upper_bound)
print ('actual min = ',df.bedroomcnt.min())
print ('actual max =', df.bedroomcnt.max())

In [ ]:
q1 = df.bathroomcnt.quantile(0.25)
q3 = df.bathroomcnt.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 + 1.15 * iqr
lower_bound =  q1 - 1.15 * iqr
print('Q1 = ' , q1 , 'Q3 =' , q3 , 'lower bound = ', lower_bound , 'upper bound = ', upper_bound)
print ('actual min = ',df.bathroomcnt.min())
print ('actual max =', df.bathroomcnt.max())

In [ ]:
q1 = df.calculatedfinishedsquarefeet.quantile(0.25)
q3 = df.calculatedfinishedsquarefeet.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 + 1.15 * iqr
lower_bound =  q1 - 1.15 * iqr
print('Q1 = ' , q1 , 'Q3 =' , q3 , 'lower bound = ', lower_bound , 'upper bound = ', upper_bound)
print ('actual min = ',df.calculatedfinishedsquarefeet.min())
print ('actual max =', df.calculatedfinishedsquarefeet.max())

In [ ]:
  cols = [
 'bathroomcnt',
 'bedroomcnt',
 'calculatedfinishedsquarefeet',
 'lotsizesquarefeet',
 'taxvaluedollarcnt',
 'logerror']

In [ ]:
df = wrangle.get_zillow_data()

In [ ]:
df.shape

In [ ]:
df = wrangle.handle_missing_values(df, prop_required_column = .9, prop_required_row = .9)

In [ ]:
df.shape

In [ ]:
df = wrangle.handle_outliers(df, cols, 1.15)

In [ ]:
df.shape

In [ ]:
df.fips.value_counts()

In [ ]:
df = w